## Import main libraries

In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense , BatchNormalization ,Dropout
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

from Predictions_test_set import *
import tensorflow as tf
import keras_tuner as kt
import keras
from tensorflow.keras.callbacks import EarlyStopping, History
from keras.preprocessing.image import ImageDataGenerator

## Import the training and test set

In this section we import both training and test sets and also an excel file built by ourselves that will help us to create a dictionary that will we used for mapping the letters with their corresponding number (0-25). Also we do some data preprocessing

In [3]:
#Import training and test sets
train=pd.read_csv("ascii-sign-language/sign_mnist_train.csv")

test = pd.read_csv("ascii-sign-language/test.csv")

#Create dictionary to map the predicted numbers (0-25) (and no cases for 9=J or 25=Z because of gesture motions).
map_letters=pd.read_excel(r"ascii-sign-language/Letters.xlsx")
map_letters["label"] = np.where(map_letters["label"] >= 9, map_letters["label"] - 1, map_letters["label"])
dictionary_letters = dict(map_letters.values)

In [4]:
x_train, x_val,y_train, y_val = train_test_split(train.iloc[:,1:] , train["label"],
                                                  stratify=train["label"], random_state=0,)

# Reshape the data to be suitable for CNN
x_train=np.array(x_train)
x_val=np.array(x_val)

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1).astype('float32')

#LETS ADJUST THE NUMBER OF CLASSES SINCE 9 IS NOT INCLUDED
y_train=np.array(y_train)
y_train = np.where(y_train >= 9, y_train - 1, y_train)
y_val=np.array(y_val)
y_val = np.where(y_val >= 9, y_val - 1, y_val)

# Normalize pixel values to be between 0 and 1
x_train /= 255
x_val /= 255

In [5]:
x_train.shape

(20591, 28, 28, 1)

In [6]:
# Create an ImageDataGenerator instance

img_augmentation = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.01,
    fill_mode='nearest',validation_split=0.2
)

## Building blocks and hyperparameter tuning

In [7]:
def model_builder1(hp):
    #HYPERPARAMETERS**************************************************
    hp_units1 = hp.Int('units1', min_value=32, max_value=162, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=162, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_pooling_strides = hp.Int('pooling_strides', min_value=1, max_value=3, step=1)  
    #MODEL************************************************************
    model = Sequential()
    model.add(Conv2D(hp_units1, (3, 3), activation=activation, padding='same',
                     input_shape=(28, 28, 1)))
    model.add(Conv2D(hp_units2, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))
    # Flatten and fully connected layers
    model.add(Flatten())
    #Last layer

    model.add(Dense(24, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])

    return model

tuner1 = kt.BayesianOptimization(model_builder1,
                     objective='val_accuracy',
                      max_trials=10,directory='hyperparameter',
                     project_name='model1')

tuner1.search_space_summary()

Search space summary
Default search space size: 5
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
pooling_strides (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}


In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
tuner1.search(img_augmentation.flow(x_train, y_train, batch_size=32), epochs=100, validation_data=(x_val, y_val),
             callbacks=[stop_early])

Trial 10 Complete [00h 03m 53s]
val_accuracy: 0.7067307829856873

Best val_accuracy So Far: 0.998251736164093
Total elapsed time: 00h 51m 47s


In [9]:
# Get the best hyperparameters
best_hps1 = tuner1.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [10]:
best_hps1

{'units1': 96,
 'units2': 96,
 'activation': 'relu',
 'learning_rate': 0.0001,
 'pooling_strides': 2}

In [75]:
best_model1 = tuner1.get_best_models(num_models=1)[0]
best_model1.save("CNN_hyper1.h5")

In [93]:
best_model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 96)        83040     
                                                                 
 batch_normalization (Batch  (None, 28, 28, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 96)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 18816)             0         
                                                                 
 dense (Dense)               (None, 24)                4

In [84]:
oracle = tuner1.oracle
best_trial_id = oracle.get_best_trials(num_trials=1)[0].trial_id

trial_info = oracle.get_trial(best_trial_id)

training_metrics = trial_info.metrics.get_history('accuracy')
validation_metrics = trial_info.metrics.get_history('val_accuracy')
training_loss = trial_info.metrics.get_history('loss')
validation_loss = trial_info.metrics.get_history('val_loss')
print("Training accuracy is ",training_metrics," and loss is ",training_loss)
print("Val accuracy is ",validation_metrics," and loss is ",validation_loss)

Training accuracy is  [MetricObservation(value=[0.9833422303199768], step=26)]  and loss is  [MetricObservation(value=[0.06356161832809448], step=26)]
Val accuracy is  [MetricObservation(value=[0.998251736164093], step=26)]  and loss is  [MetricObservation(value=[0.014613224193453789], step=26)]


In [32]:
def model_builder2(hp):
    #HYPERPARAMETERS
    hp_units1 = hp.Int('units1', min_value=32, max_value=162, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=162, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=162, step=32)
    hp_units4 = hp.Int('units4', min_value=32, max_value=162, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_pooling_strides = hp.Int('pooling_strides', min_value=1, max_value=3, step=1)  

    #MODEL
    model = Sequential()
    model.add(Conv2D(hp_units1, (3, 3), activation=activation, padding='same',
                     input_shape=(28, 28, 1)))
    model.add(Conv2D(hp_units2, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))

    model.add(Conv2D(hp_units3, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units4, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))

    # Flatten and fully connected layers
    model.add(Flatten())
    #Last layer
    model.add(BatchNormalization())
    model.add(Dense(24, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])

    return model

tuner2 = kt.BayesianOptimization(model_builder2,
                     objective='val_accuracy',
                      max_trials=10,directory='hyperparameter',
                     project_name='model2')

tuner2.search_space_summary()

Search space summary
Default search space size: 7
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
pooling_strides (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}


In [33]:
tuner2.search(img_augmentation.flow(x_train, y_train, batch_size=32), epochs=100, validation_data=(x_val, y_val),
             callbacks=[stop_early])

Trial 10 Complete [00h 04m 06s]
val_accuracy: 0.941724956035614

Best val_accuracy So Far: 0.995192289352417
Total elapsed time: 00h 28m 34s


In [34]:
best_hps2 = tuner2.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
best_hps2

{'units1': 32,
 'units2': 64,
 'units3': 160,
 'units4': 160,
 'activation': 'relu',
 'learning_rate': 0.001,
 'pooling_strides': 3}

In [76]:
best_model2 = tuner2.get_best_models(num_models=1)[0]
best_model2.save("CNN_hyper2.h5")

In [92]:
best_model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization (Batch  (None, 28, 28, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 9, 9, 64)          0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 160)         92320     
                                                                 
 batch_normalization_1 (Bat  (None, 9, 9, 160)         6

In [83]:

oracle = tuner2.oracle

best_trial_id = oracle.get_best_trials(num_trials=1)[0].trial_id

trial_info = oracle.get_trial(best_trial_id)
training_metrics = trial_info.metrics.get_history('accuracy')
validation_metrics = trial_info.metrics.get_history('val_accuracy')
training_loss = trial_info.metrics.get_history('loss')
validation_loss = trial_info.metrics.get_history('val_loss')
print("Training accuracy is ",training_metrics," and loss is ",training_loss)
print("Val accuracy is ",validation_metrics," and loss is ",validation_loss)

Training accuracy is  [MetricObservation(value=[0.9827594757080078], step=8)]  and loss is  [MetricObservation(value=[0.05249616876244545], step=8)]
Val accuracy is  [MetricObservation(value=[0.995192289352417], step=8)]  and loss is  [MetricObservation(value=[0.013153145089745522], step=8)]


In [37]:
def model_builder3(hp):
    #HYPERPARAMETERS
    hp_units1 = hp.Int('units1', min_value=32, max_value=162, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=162, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=162, step=32)
    hp_units4 = hp.Int('units4', min_value=32, max_value=162, step=32)
    hp_units5 = hp.Int('units5', min_value=32, max_value=162, step=32)
    hp_units6 = hp.Int('units6', min_value=32, max_value=162, step=32)
    activation = hp.Choice("activation", ["relu", "tanh"])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_pooling_strides = hp.Int('pooling_strides', min_value=1, max_value=3, step=1)  

    #MODEL
    model = Sequential()
    model.add(Conv2D(hp_units1, (3, 3), activation=activation, padding='same',
                     input_shape=(28, 28, 1)))
    model.add(Conv2D(hp_units2, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))

    model.add(Conv2D(hp_units3, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units4, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))

    model.add(Conv2D(hp_units5, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units6, (3, 3), activation=activation, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(hp_pooling_strides, hp_pooling_strides) ))

    # Flatten and fully connected layers
    model.add(Flatten())
    #Last layer
    model.add(BatchNormalization())
    model.add(Dropout(0.05))
    model.add(Dense(24, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])

    return model

tuner3 = kt.BayesianOptimization(model_builder3,
                     objective='val_accuracy',
                      max_trials=10,directory='hyperparameter',
                     project_name='model3')

tuner3.search_space_summary()

Search space summary
Default search space size: 9
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units6 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
pooling_strides (Int)
{'defau

In [38]:
tuner3.search(img_augmentation.flow(x_train, y_train, batch_size=32), epochs=100, validation_data=(x_val, y_val),
             callbacks=[stop_early])

Trial 10 Complete [00h 03m 25s]
val_accuracy: 0.9784382581710815

Best val_accuracy So Far: 0.9992715716362
Total elapsed time: 00h 37m 39s


In [39]:
best_hps3 = tuner3.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
best_hps3

{'units1': 160,
 'units2': 64,
 'units3': 128,
 'units4': 96,
 'units5': 96,
 'units6': 64,
 'activation': 'relu',
 'learning_rate': 0.0001,
 'pooling_strides': 1}

In [40]:
best_model3 = tuner3.get_best_models(num_models=1)[0]
best_model3.save("CNN_hyper3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [91]:
best_model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 160)       1600      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        92224     
                                                                 
 batch_normalization (Batch  (None, 28, 28, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 128)       73856     
                                                                 
 batch_normalization_1 (Bat  (None, 27, 27, 128)       5

In [85]:

oracle = tuner3.oracle

best_trial_id = oracle.get_best_trials(num_trials=1)[0].trial_id

trial_info = oracle.get_trial(best_trial_id)
training_metrics = trial_info.metrics.get_history('accuracy')
validation_metrics = trial_info.metrics.get_history('val_accuracy')
training_loss = trial_info.metrics.get_history('loss')
validation_loss = trial_info.metrics.get_history('val_loss')
print("Training accuracy is ",training_metrics," and loss is ",training_loss)
print("Val accuracy is ",validation_metrics," and loss is ",validation_loss)

Training accuracy is  [MetricObservation(value=[0.9896070957183838], step=17)]  and loss is  [MetricObservation(value=[0.04822712764143944], step=17)]
Val accuracy is  [MetricObservation(value=[0.9992715716362], step=17)]  and loss is  [MetricObservation(value=[0.0020518091041594744], step=17)]


In [44]:
def model_builder4(hp):
    #HYPERPARAMETERS
    hp_units1 = hp.Int('units1', min_value=32, max_value=162, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=162, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=162, step=32)
    hp_units4 = hp.Int('units4', min_value=32, max_value=162, step=32)
    hp_units5 = hp.Int('units5', min_value=32, max_value=162, step=32)
    hp_units6 = hp.Int('units6', min_value=32, max_value=162, step=32)
    hp_units7 = hp.Int('units7', min_value=32, max_value=162, step=32)
    hp_units8 = hp.Int('units8', min_value=32, max_value=162, step=32)

    activation1 = hp.Choice("activation1", ["relu", "tanh"])
    activation2 = hp.Choice("activation2", ["relu", "tanh"])
    activation3 = hp.Choice("activation3", ["relu", "tanh"])
    activation4 = hp.Choice("activation4", ["relu", "tanh"])
    activation5 = hp.Choice("activation5", ["relu", "tanh"])
    activation6 = hp.Choice("activation6", ["relu", "tanh"])
    activation7 = hp.Choice("activation7", ["relu", "tanh"])
    activation8 = hp.Choice("activation8", ["relu", "tanh"])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_pooling_strides = hp.Int('pooling_strides', min_value=1, max_value=3, step=1)  

    #MODEL
    model = Sequential()
    model.add(Conv2D(hp_units1, (3, 3), activation=activation1, padding='same',input_shape=(28, 28, 1)))
    model.add(Conv2D(hp_units2, (3, 3), activation=activation2, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2) ))

    model.add(Conv2D(hp_units3, (3, 3), activation=activation3, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units4, (3, 3), activation=activation4, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2) ))

    model.add(Conv2D(hp_units5, (3, 3), activation=activation5, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units6, (3, 3), activation=activation6, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2) ))

    model.add(Conv2D(hp_units7, (3, 3), activation=activation7, padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(hp_units8, (3, 3), activation=activation8, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), strides=(2, 2) ))

    # Flatten and fully connected layers
    model.add(Flatten())
    #Last layer
    model.add(BatchNormalization())
    model.add(Dropout(0.05))
    model.add(Dense(24, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])

    return model

tuner4 = kt.BayesianOptimization(model_builder4,
                     objective='val_accuracy',
                      max_trials=20,directory='hyperparameter',
                     project_name='model4')

tuner4.search_space_summary()

Reloading Tuner from hyperparameter/model4/tuner0.json
Search space summary
Default search space size: 18
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units6 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units7 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 162, 'step': 32, 'sampling': 'linear'}
units8 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'm

In [45]:
tuner4.search(img_augmentation.flow(x_train, y_train, batch_size=32), epochs=100, validation_data=(x_val, y_val),
             callbacks=[stop_early])

Trial 20 Complete [00h 03m 51s]
val_accuracy: 0.7642773985862732

Best val_accuracy So Far: 1.0
Total elapsed time: 01h 05m 26s


In [46]:
best_hps4 = tuner4.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
best_hps4

{'units1': 128,
 'units2': 96,
 'units3': 128,
 'units4': 128,
 'units5': 64,
 'units6': 128,
 'units7': 32,
 'units8': 160,
 'activation1': 'tanh',
 'activation2': 'relu',
 'activation3': 'tanh',
 'activation4': 'relu',
 'activation5': 'relu',
 'activation6': 'relu',
 'activation7': 'relu',
 'activation8': 'relu',
 'learning_rate': 0.0001,
 'pooling_strides': 3}

In [47]:
best_model4 = tuner4.get_best_models(num_models=1)[0]
best_model4.save("CNN_hyper4.h5")

In [90]:
best_model4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 96)        110688    
                                                                 
 batch_normalization (Batch  (None, 28, 28, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 96)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       110720    
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 128)       5

In [86]:

oracle = tuner4.oracle

best_trial_id = oracle.get_best_trials(num_trials=1)[0].trial_id

trial_info = oracle.get_trial(best_trial_id)
training_metrics = trial_info.metrics.get_history('accuracy')
validation_metrics = trial_info.metrics.get_history('val_accuracy')
training_loss = trial_info.metrics.get_history('loss')
validation_loss = trial_info.metrics.get_history('val_loss')
print("Training accuracy is ",training_metrics," and loss is ",training_loss)
print("Val accuracy is ",validation_metrics," and loss is ",validation_loss)

Training accuracy is  [MetricObservation(value=[0.9966976046562195], step=13)]  and loss is  [MetricObservation(value=[0.015952402725815773], step=13)]
Val accuracy is  [MetricObservation(value=[1.0], step=13)]  and loss is  [MetricObservation(value=[0.0018107733922079206], step=13)]


In [77]:
predictions1 = best_model1.predict(x_val)
predictions2 = best_model2.predict(x_val)
predictions3 = best_model3.predict(x_val)
predictions4 = best_model4.predict(x_val)

final_pred1 = make_predictions_test(test , best_model1 , dictionary_letters)
final_pred1.to_csv("hyper_CNN1.csv",index=False)

final_pred2 = make_predictions_test(test , best_model2 , dictionary_letters)
final_pred2.to_csv("hyper_CNN2.csv",index=False)

final_pred3 = make_predictions_test(test , best_model3 , dictionary_letters)
final_pred3.to_csv("hyper_CNN3.csv",index=False)

final_pred4 = make_predictions_test(test , best_model4 , dictionary_letters)
final_pred4.to_csv("hyper_CNN4.csv",index=False)

47/94 [==============>...............] - ETA: 0s

/content/Predictions_test_set.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"label"] = y_pred


17/94 [====>.........................] - ETA: 0s

/content/Predictions_test_set.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"label"] = y_pred


38/94 [===========>..................] - ETA: 0s

/content/Predictions_test_set.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"label"] = y_pred


94/94 [==============================] - 0s 3ms/step


/content/Predictions_test_set.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"label"] = y_pred


## Resources:

* https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

* https://keras.io/guides/keras_tuner/getting_started/

* https://keras.io/api/layers/pooling_layers/max_pooling2d/
* https://www.tensorflow.org/tutorials/keras/keras_tuner
* https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator